使用許諾条件

①	設計一次エネルギー消費量算定法に基づく計算方法をPythonによりコーディングし、Jupyter notebook形式で保存したファイル（以下、本ファイルと呼ぶ）の著作権は、一般社団法人日本サステナブル建築協会に帰属します。
Copyright©2019. 一般社団法人日本サステナブル建築協会
All rights reserved. 

②	本ファイルは、住戸の設計一次エネルギー消費量算定方法に係る「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）2.1算定方法（第十一章第二節 日射に関する地域の区分と日射量等）」に示す計算方法に基づき、国土交通省国土技術政策総合研究所・国立研究開発法人建築研究所、株式会社ユーワークス、株式会社住環境計画研究所が作成したものです。

③	本ファイルは、計算方法に示す計算過程の確認及び調査研究等に活用することを目的として作成されたものであり、一般社団法人日本サステナブル建築協会は著作権について放棄はしませんが、使用する各人の自己責任において仕様を変更することを妨げるものではありません。ただし、本ファイルによる計算結果を住宅の省エネルギー基準で規定する設計一次エネルギー消費量の結果として公的な申請に用いることはできません。

④	本ファイルを変更するかしないかを問わず、以下の条件を満たす場合に限り、頒布および使用が許可されます。
1.	頒布物の付属ドキュメント等の資料に、上記の著作権表示、本条件一覧、および下記免責条項を含めること。
2.	書面による特別の許可なしに、本ファイルから派生した製品の宣伝または販売の促進に、一般社団法人日本サステナブル建築協会、国土技術政策総合研究所・国立研究開発法人建築研究所および株式会社ユーワークス、株式会社住環境計画研究所の名前またはコントリビューターの名前を使用してはならない。

⑤	本ファイルは、著作権者およびコントリビューターによって「現状のまま」提供されており、明示黙示を問わず、商業的な使用可能性、および特定の目的に対する適合性に関する暗黙の保証も含め、またそれに限定されない、いかなる保証もありません。著作権もコントリビューターも、事由のいかんを問わず、損害発生の原因いかんを問わず、かつ責任の根拠が契約であるか厳格責任であるか（過失その他の）不法行為であるかを問わず、仮にそのような損害が発生する可能性を知らされていたとしても、本ファイルの使用によって発生した（代用品または代用サービスの調達、使用の喪失、データの喪失、利益の喪失、業務の中断も含め、またそれに限定されない）直接損害、間接損害、偶発的な損害、特別損害、懲罰的損害、または結果損害について、一切の責任を負わないものとします。



In [ ]:
import numpy as np
import pandas as pd
from functools import lru_cache

## 第二節　日射に関する地域区分と日射量 / Section 2 Regional division and amount of solar radiation

### 1．適用範囲 / Scope of application

省略

ommision

### 2. 引用規格 / Cited standard

省略

ommision

### 3. 用語の定義 / Definition of terms

省略

ommision

### 4．年間日射量地域区分 / Annual solar radiation area division

省略

ommision

### 5．日射量等 / Solar radiation amount

省略

ommision

## 付録 A 傾斜面における単位面積当たりの平均日射量 / Appendix A Average amount of solar radiation per unit area on inclined surface

### A.1 記号及び単位 / Symbol and unit

省略

ommision

### A.2 傾斜面における単位面積当たりの平均日射量の計算方法 / Calculation method of average solar radiation amount per unit area on inclined surface

In [ ]:
@lru_cache()
def load_solrad(region, sol_region):
    """日射量データの読み込み

    :param region: 省エネルギー地域区分
    :type region: int
    :param sol_region: 年間日射地域区分(1-5)
    :type sol_region: int
    :return: 日射量データ
    :rtype: DateFrame
    """
    posnum = get_position_num(region, sol_region)
    return pd.read_csv('data/solar/%s.csv' % posnum, skiprows=2, nrows=24 * 365,
                       names=('T_ex', 'I_DN', 'I_Sky', 'h', 'A'))


def get_position_num(region, sol_region):
    """地域区分、日射地域区分から地域番号

    :param region: 省エネルギー地域区分
    :type region: int
    :param sol_region: 年間日射地域区分(1-5)
    :type sol_region: int
    :return: 地域区分、日射地域区分から地域番号
    :rtype: str
    """
    nums = [
        ['7', '117', '124', '-', '-'],
        ['49', '63', '59', '-', '-'],
        ['190', '230', '426', '403', '412'],
        ['286', '186', '292', '423', '401'],
        ['593', '542', '495', '473', '420'],
        ['-', '569', '551', '480', '438'],
        ['819-A1', '819-A2', '819', '798', '797'],
        ['-', '-', '826', '836', '842']
    ]
    return nums[region - 1][sol_region - 1]


In [ ]:
def get_Theta_ex(df):
    """外気温度

    :param df: 気温[℃],絶湿[g/kg']
    :type df: DataFrame
    :return: 外気温度
    :rtype: ndarray
    """
    return df['T_ex'].values


$$\large
\displaystyle
I_{s,d,t} = \left\{ \begin{array}{ll}
    I_{D,d,t} + I_{d,d,t} & (I_{D,d,t} \geq 0)\\
    I_{d,d,t} & (I_{D,d,t} < 0)\\
    \end{array}
\right.
$$

In [ ]:
def calc_I_s_d_t(P_alpha, P_beta, df):
    """傾斜面の単位面積当たりの平均日射量

    :param P_alpha: 方位角 (°)
    :type P_alpha: float
    :param P_beta: 傾斜角 (°)
    :type P_beta: float
    :param df: load_solrad の返り値
    :type df: DateFrame
    :return: 傾斜面の単位面積当たりの平均日射量
    :rtype: ndarray
    """
    I_DN_d_t = df['I_DN'].values / 3.6 * 1000
    I_Sky_d_t = df['I_Sky'].values / 3.6 * 1000
    h_d_t = np.radians(df['h'].values)
    A_d_t = np.radians(df['A'].values * (df['A'].values >= 0) + (df['A'].values + 360.0) * (df['A'].values < 0))

    I_D_d_t = get_I_D_d_t(I_DN_d_t, h_d_t, P_alpha, P_beta, A_d_t)
    I_d_d_t = get_I_d_d_t(I_Sky_d_t, P_beta)

    return I_D_d_t * (I_D_d_t >= 0) + I_d_d_t


$$\large
I_{D,d,t} = I_{DN,d,t} \{sin\ h_{d,t}\ cos\ P_{\beta} + cos\ h_{d,t}\ sin\ P_{\beta}\ cos(P_{\alpha} - A_{d,t})\}
$$

In [ ]:
def get_I_D_d_t(I_DN_d_t, h_d_t, P_alpha, P_beta, A_d_t):
    """傾斜面の単位面積当たりの直達日射量

    :param I_DN_d_t: 法線面直達日射量
    :type I_DN_d_t: ndarray
    :param h_d_t: 太陽高度
    :type h_d_t: ndarray
    :param P_alpha: 方位角 (°)
    :type P_alpha: float
    :param P_beta: 傾斜角 (°)
    :type P_beta: float
    :param A_d_t: 太陽方位角 (°)
    :type A_d_t: ndarray
    :return: 傾斜面の単位面積当たりの直達日射量
    :rtype: ndarray
    """
    return I_DN_d_t * (np.sin(h_d_t) * np.cos(P_beta) + np.cos(h_d_t) * np.sin(P_beta) * np.cos(P_alpha - A_d_t))


$$\large
I_{d,d,t} = I_{Sky,d,t} \frac{(1+cos\ P_{\beta})}{2}
$$

$I_{s,d,t}$：日付$d$の時刻$t$における傾斜面の単位面積当たりの平均日射量(W/m<sup>2</sup>) / Average amount of solar radiation per unit area of inclined surface at time $ t $ of date $ d $  
$I_{D,d,t}$：日付$d$の時刻$t$における傾斜面の単位面積当たりの直達日射量(W/m<sup>2</sup>) / Direct solar radiation per unit area of inclined surface at time $ t $ of date $ d $  
$I_{d,d,t}$：日付$d$の時刻$t$における傾斜面の単位面積当たりの天空放射量(W/m<sup>2</sup>) / Sky radiation amount per unit area of inclined plane at time $ t $ of date $ d $  
$I_{DN,d,t}$：日付$d$の時刻$t$における法線面直達日射量(W/m<sup>2</sup>) / Normal surface direct solar radiation at time $ t $ of date $ d $  
$I_{Sky,d,t}$：日付$d$の時刻$t$における水平面天空日射量(W/m<sup>2</sup>) / Horizontal sky insolation at time $ t $ of date $ d $  
$h_{d,t}$：日付$d$の時刻$t$における太陽高度(°) / Solar altitude at time $ t $ with date $ d $  
$A_{d,t}$：日付$d$の時刻$t$における太陽方位角(°) / Sun azimuth at time $ t $ of date $ d $  
$P_{\alpha}$：傾斜面の方位角(°) / Azimuth of inclined plane  
$P_{\beta}$：傾斜面の傾斜角(°) / Inclination angle of inclined surface  

In [ ]:
def get_I_d_d_t(I_Sky_d_t, P_beta):
    """傾斜面の単位面積当たりの天空放射量

    :param I_Sky_d_t: 水平面天空日射量
    :type I_Sky_d_t: ndarray
    :param P_beta: 傾斜角 (°)
    :type P_beta: float
    :return: 傾斜面の単位面積当たりの天空放射量
    :rtype: ndarray
    """
    return I_Sky_d_t * ((1.0 + np.cos(P_beta)) / 2.0)


<div style="text-align: center;font-weight: bold;">表　傾斜面の方位角<br>Table: Azimuth of inclined plane</div>

<table>
<thead>
<tr><th>方位角の範囲<br>Azimuth range</th><th>丸めた値<br>Rounded value</th><th>方位角の範囲<br>Azimuth range</th><th>丸めた値<br>Rounded value</th></tr>
</thead>
<tbody>
<tr><th>－15度（345度）から15度<br>-15 degrees (345 degrees) to 15 degrees</th><td>0度<br>0 degrees</td><th>165度から195度<br>165 degrees to 195 degrees</th><td>180度<br>180 degrees</td></tr>
<tr><th>15度から45度<br>15 degrees to 45 degrees</th><td>30度<br>30 degrees</td><th>195度から225度<br>195 degrees to 225 degrees</th><td>210度<br>210 degrees</td></tr>
<tr><th>45度から75度<br>45 degrees to 75 degrees</th><td>60度<br>60 degrees</td><th>225度から255度<br>225 degrees to 255 degrees</th><td>240度<br>240 degrees</td></tr>
<tr><th>75度から105度<br>75 degrees to 105 degrees</th><td>90度<br>90 degrees</td><th>255度から285度<br>255 degrees to 285 degrees</th><td>270度<br>270 degrees</td></tr>
<tr><th>105度から135度<br>105 degrees to 135 degrees</th><td>120度<br>120 degrees</td><th>285度から315度<br>285 degrees to 315 degrees</th><td>300度<br>300 degrees</td></tr>
<tr><th>135度から165度<br>135 degrees to 165 degrees</th><td>150度<br>150 degrees</td><th>315度から345度<br>315 degrees to 345 degrees</th><td>330度<br>330 degrees</td></tr>
</tbody>
</table>

傾斜面の傾斜角$P_{\beta}$は、水平面からの傾斜面の角度であり、傾斜面の傾斜角の一の位を四捨五入した値とし、90度を超える場合は90度とする。

The inclination angle $ P_{\beta} $ of the inclined surface is the angle of the inclined surface from the horizontal plane, and is taken as the value obtained by rounding off one place of the inclination angle of the inclined surface.